In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, classification_report
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib import pyplot
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import seaborn as sns
from sklearn.svm import SVC
from sklearn_evaluation import plot
import seaborn
from math import sqrt
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
import smote_variants as sv
from adasyn import ADASYN
from sklearn.model_selection import StratifiedKFold

In [ ]:
#Coal rank classification
#classification algo- SVM, RF, XGB, Logistic regression

In [16]:
xls = pd.ExcelFile('Reflectance-rank-2.06.2020.xlsx')
#xls = pd.ExcelFile('train-test.xlsx')
dfTotal = pd.read_excel(xls, 'Reflc-rank')
(unique, counts) = np.unique(dfTotal.iloc[:,-1], return_counts = True)
for u,c in zip(unique, counts):
    print(u,c)
(dfTotal.shape)

HV-A Bituminous 30
HV-B Bituminous 5
HV-C Bituminous 10
LV Bituminous 102
Lignite-A 6
MV Bituminous 34
Semi-anthracite 6
Subbituminous-A 8
Subbituminous-B 4
Subbituminous-C 7


(212, 2103)

In [3]:
# dfTest= pd.read_excel(xls, 'Reflc-rank-test')
# (unique, counts) = np.unique(dfTest.iloc[:,-1], return_counts = True)
# for u,c in zip(unique, counts):
#     print(u,c)
# (dfTest.shape)

In [4]:
# dfTotal = pd.concat([dfTrain, dfTest], axis = 0)
# (unique, counts) = np.unique(dfTotal.iloc[:,-1], return_counts = True)
# #skf = StratifiedKFold(n_splits=5)
# # for train_index, test_index in skf.split(dfTotal.iloc[:,:-1], dfTotal.iloc[:,-1]):
# #     print("TRAIN:", train_index, "TEST:", test_index)
# for u,c in zip(unique, counts):
#     print(u,c)
# (dfTotal.shape)


In [5]:
def SVM(df1, df_id, class_weight = None, impute = True):
    param_grid = { 
         'C': [0.01, 0.1, 1, 100, 200],
         'kernel' : ['rbf', 'linear'],
        'gamma': ['scale', 'auto', 0.001, 0.01, 1, 10, 50],

    }
    enc = LabelEncoder()
    y = df1.iloc[:,-1:].values
    enc.fit(y)
    y = enc.transform(y)
    print(list(enc.classes_))
    X = StandardScaler().fit_transform(df1.iloc[:,1:-1])
    X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.2)
    if impute:
        oversampler= sv.MulticlassOversampling(sv.distance_SMOTE(n_neighbors = 10))
        X_train, y_train = oversampler.sample(X_train, y_train)

#         adsn = ADASYN(k=10,imb_threshold=0.5, ratio=0.75)
#         X_train, y_train = adsn.fit_transform(X_train, y_train)
    skf = StratifiedKFold(n_splits=5)
    if impute:
        pca = PCA(n_components = 500)
        x_train = pca.fit_transform(X_train)
        x_test = pca.transform(X_test)
    else:
        pca = PCA(n_components = X_train.shape[0])
        x_train = pca.fit_transform(X_train)
        x_test = pca.transform(X_test)
        #x_train, x_test = X_train, X_test
        
    clf = GridSearchCV(SVC(class_weight = class_weight),
                       param_grid, cv = skf, n_jobs = -1, scoring = 'f1_macro')
    #clf = RandomForestClassifier(max_depth=500, n_estimators=10000)
    clf.fit(x_train, y_train)
    model = clf.best_estimator_
    print(clf.best_params_)
    grid_scores = clf.cv_results_
    model.fit(x_train, y_train)
    
    y_res = model.predict(x_test)
    
    y_res_train = model.predict(x_train)
    
    total, correct = 0,0
    for y_ac, y_pred in zip(y_test, y_res):
        if y_ac == y_pred:
            correct = correct + 1
        total = total + 1
    print('Accuracy: ' + str(correct) + str('/') + str(total) + str(' = ') + str(correct/total*100)) 
    
#     true1, true2, true3, true4, true5, true6, true7, true8, true9, true0 = 0,0, 0,0,0,0,0,0,0,0
#     tot1, tot2, tot3, tot4, tot5, tot6, tot7, tot8, tot9, tot0 = 0,0, 0,0,0,0,0,0,0,0

#     for y_ac, y_pred in zip(y_test, y_res):
#         if y_ac == 0:
#             tot0 = tot0+1
#             if y_ac == y_pred:
#                 true0 = true0 + 1
#         if y_ac == 1:
#             tot1 = tot1+1
#             if y_ac == y_pred:
#                 true1 = true1 + 1

#         if y_ac == 2:
#             tot2 = tot2+1
#             if y_ac == y_pred:
#                 true2 = true2 + 1
#         if y_ac == 3:
#             tot3 = tot3+1
#             if y_ac == y_pred:
#                 true3 = true3 + 1
#         if y_ac == 4:
#             tot4 = tot4+1
#             if y_ac == y_pred:
#                 true4 = true4 + 1

#         if y_ac == 0:
#             tot5 = tot5+1
#             if y_ac == y_pred:
#                 true5 = true5 + 1
#         if y_ac == 6:
#             tot6 = tot6+1
#             if y_ac == y_pred:
#                 true6 = true6 + 1
#         if y_ac == 7:
#             tot7 = tot7+1
#             if y_ac == y_pred:
#                 true7 = true7 + 1

#         if y_ac == 8:
#             tot8 = tot8+1
#             if y_ac == y_pred:
#                 true8 = true8 + 1
#         if y_ac == 9:
#             tot9 = tot9+1
#             if y_ac == y_pred:
#                 true9 = true9 + 1
                
#     tot = tot0 + tot1 +tot2 +tot3 +tot4 +tot5+tot6+tot7+tot8+tot9
#     true = true0 + true1 + true2 + true3 + true4 + true5 + true6 + true7 + true8 + true9
#     print('Test Accuracy: ' + str(true) + str('/') + str(tot) + str(' = ') + str(true/tot*100))
#     print('HV-A Bituminous: ' + str(true0) + str('/') + str(tot0) + str(' = ') + str(true0/tot0*100))
#     print('HV-B Bituminous: ' + str(true1) + str('/') + str(tot1) + str(' = ') + str(true1/tot1*100))
#     print('HV-C Bituminous: ' + str(true2) + str('/') + str(tot2) + str(' = ') + str(true2/tot2*100))
#     print('LV Bituminous: ' + str(true3) + str('/') + str(tot3) + str(' = ') + str(true3/tot3*100))
#     print('Lignite-A: ' + str(true4) + str('/') + str(tot4) + str(' = ') + str(true4/tot4*100))
#     print('MV Bituminous: ' + str(true5) + str('/') + str(tot5) + str(' = ') + str(true5/tot5*100))
#     print('Semi-anthracite: ' + str(true6) + str('/') + str(tot6) + str(' = ') + str(true6/tot6*100))
#     print('SubBituminous-A: ' + str(true7) + str('/') + str(tot7) + str(' = ') + str(true7/tot7*100))
#     print('SubBituminous-B: ' + str(true8) + str('/') + str(tot8) + str(' = ') + str(true8/tot8*100))
#     print('SubBituminous-C: ' + str(true9) + str('/') + str(tot9) + str(' = ') + str(true9/tot9*100))
    temp_np1 = np.vstack((y_train, y_res_train))
    temp_np2 = np.vstack((y_test, y_res))
    temp_np = np.hstack((temp_np1, temp_np2))
    df_temp = pd.DataFrame(temp_np2)
    filename = str(df_id) + '_' + 'svm_' + str(impute) + str(round(correct/total*100,2))+ '.csv'
    df_temp.to_csv('./results/' + filename,index=False)

In [10]:
def random_forest(df1, df_id, class_weight = None, impute = True):
    param_grid = {
                'n_estimators': [1000, 2000], 
                'max_depth' : [20, 5, 10],
                'min_samples_split' : [3], 
                'min_samples_leaf' : [2], 
                'max_features' : [.5]
                }
    enc = LabelEncoder()
    y = df1.iloc[:,-1:].values
    enc.fit(y)
    y = enc.transform(y)
    print(list(enc.classes_))
    X = StandardScaler().fit_transform(df1.iloc[:,1:-1])
    X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.2)
    if impute:
        oversampler= sv.MulticlassOversampling(sv.distance_SMOTE(n_neighbors = 10))
        X_train, y_train = oversampler.sample(X_train, y_train)

#         adsn = ADASYN(k=10,imb_threshold=0.5, ratio=0.75)
#         X_train, y_train = adsn.fit_transform(X_train, y_train)

    skf = StratifiedKFold(n_splits=5)
    if impute:
        pca = PCA(n_components = 500)
        x_train = pca.fit_transform(X_train)
        x_test = pca.transform(X_test)
    else:
        print(X_train.shape[0])
        pca = PCA(n_components = X_train.shape[0])
        x_train = pca.fit_transform(X_train)
        x_test = pca.transform(X_test)
        
    clf = GridSearchCV(RandomForestClassifier(class_weight = class_weight),
                       param_grid, cv = skf, n_jobs = -1, scoring = 'f1_macro')
    #clf = RandomForestClassifier(max_depth=500, n_estimators=10000)
    clf.fit(x_train, y_train)
    model = clf.best_estimator_
    print(clf.best_params_)
    grid_scores = clf.cv_results_
    model.fit(x_train, y_train)
    y_res = model.predict(x_test)
    
    y_res_train = model.predict(x_train)
    total, correct = 0,0
    for y_ac, y_pred in zip(y_test, y_res):
        if y_ac == y_pred:
            correct = correct + 1
        total = total + 1
    print('Accuracy: ' + str(correct) + str('/') + str(total) + str(' = ') + str(correct/total*100)) 
    
    
#     true1, true2, true3, true4, true5, true6, true7, true8, true9, true0 = 0,0, 0,0,0,0,0,0,0,0
#     tot1, tot2, tot3, tot4, tot5, tot6, tot7, tot8, tot9, tot0 = 0,0, 0,0,0,0,0,0,0,0

#     for y_ac, y_pred in zip(y_test, y_res):
#         if y_ac == 0:
#             tot0 = tot0+1
#             if y_ac == y_pred:
#                 true0 = true0 + 1
#         if y_ac == 1:
#             tot1 = tot1+1
#             if y_ac == y_pred:
#                 true1 = true1 + 1

#         if y_ac == 2:
#             tot2 = tot2+1
#             if y_ac == y_pred:
#                 true2 = true2 + 1
#         if y_ac == 3:
#             tot3 = tot3+1
#             if y_ac == y_pred:
#                 true3 = true3 + 1
#         if y_ac == 4:
#             tot4 = tot4+1
#             if y_ac == y_pred:
#                 true4 = true4 + 1

#         if y_ac == 0:
#             tot5 = tot5+1
#             if y_ac == y_pred:
#                 true5 = true5 + 1
#         if y_ac == 6:
#             tot6 = tot6+1
#             if y_ac == y_pred:
#                 true6 = true6 + 1
#         if y_ac == 7:
#             tot7 = tot7+1
#             if y_ac == y_pred:
#                 true7 = true7 + 1

#         if y_ac == 8:
#             tot8 = tot8+1
#             if y_ac == y_pred:
#                 true8 = true8 + 1
#         if y_ac == 9:
#             tot9 = tot9+1
#             if y_ac == y_pred:
#                 true9 = true9 + 1
                
#     tot = tot0 + tot1 +tot2 +tot3 +tot4 +tot5+tot6+tot7+tot8+tot9
#     true = true0 + true1 + true2 + true3 + true4 + true5 + true6 + true7 + true8 + true9
#     print('Test Accuracy: ' + str(true) + str('/') + str(tot) + str(' = ') + str(true/tot*100))
#     print('HV-A Bituminous: ' + str(true0) + str('/') + str(tot0) + str(' = ') + str(true0/tot0*100))
#     print('HV-B Bituminous: ' + str(true1) + str('/') + str(tot1) + str(' = ') + str(true1/tot1*100))
#     print('HV-C Bituminous: ' + str(true2) + str('/') + str(tot2) + str(' = ') + str(true2/tot2*100))
#     print('LV Bituminous: ' + str(true3) + str('/') + str(tot3) + str(' = ') + str(true3/tot3*100))
#     print('Lignite-A: ' + str(true4) + str('/') + str(tot4) + str(' = ') + str(true4/tot4*100))
#     print('MV Bituminous: ' + str(true5) + str('/') + str(tot5) + str(' = ') + str(true5/tot5*100))
#     print('Semi-anthracite: ' + str(true6) + str('/') + str(tot6) + str(' = ') + str(true6/tot6*100))
#     print('SubBituminous-A: ' + str(true7) + str('/') + str(tot7) + str(' = ') + str(true7/tot7*100))
#     print('SubBituminous-B: ' + str(true8) + str('/') + str(tot8) + str(' = ') + str(true8/tot8*100))
#     print('SubBituminous-C: ' + str(true9) + str('/') + str(tot9) + str(' = ') + str(true9/tot9*100))
    temp_np1 = np.vstack((y_train, y_res_train))
    temp_np2 = np.vstack((y_test, y_res))
    temp_np = np.hstack((temp_np1, temp_np2))
    df_temp = pd.DataFrame(temp_np2)
    filename = str(df_id) + '_' + 'RF_' + str(impute) + str(round(correct/total*100,2))+ '.csv'
    df_temp.to_csv('./results/' + filename,index=False)


In [7]:
def XGB(df1, class_weight = None, impute = True):
    param_grid =  {
                  'learning_rate': [.001], #so called `eta` value
                  'max_depth': [3, 5, 8],
                  'min_child_weight': [2, 5],
                  'subsample': [0.7],
                  'colsample_bytree': [0.8],
                  'n_estimators': [500, 1000, 2000]
                  }
    enc = LabelEncoder()
    y = df1.iloc[:,-1:].values
    enc.fit(y)
    y = enc.transform(y)
    print(list(enc.classes_))
    X = StandardScaler().fit_transform(df1.iloc[:,1:-1])
    X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.3)
    if impute:
        oversampler= sv.MulticlassOversampling(sv.distance_SMOTE(n_neighbors = 10))
        X_train, y_train = oversampler.sample(X_train, y_train)

#         adsn = ADASYN(k=10,imb_threshold=0.5, ratio=0.75)
#         X_train, y_train = adsn.fit_transform(X_train, y_train)  # your imbalanced dataset is in X,y
       
    skf = StratifiedKFold(n_splits=5)
    if impute:
        pca = PCA(n_components = 500)
        x_train = pca.fit_transform(X_train)
        x_test = pca.transform(X_test)
    else:
        print(X_train.shape[0])
        pca = PCA(n_components = X_train.shape[0])
        x_train = pca.fit_transform(X_train)
        x_test = pca.transform(X_test)
        

    clf = GridSearchCV(xgb.XGBRegressor(objective = 'multi:softmax', num_class = 10, nthread = 4, tree_method = 'gpu_hist', eval_metric = 'merror')
    , param_grid, cv = skf, n_jobs = -1, scoring = 'f1_macro')
    #clf = RandomForestClassifier(max_depth=500, n_estimators=10000)
    clf.fit(x_train, y_train)
    model = clf.best_estimator_
    print(clf.best_params_)
    grid_scores = clf.cv_results_
    model.fit(x_train, y_train)
    y_res = model.predict(x_test)
    y_res_train = model.predict(x_train)
    total, correct = 0,0
    for y_ac, y_pred in zip(y_train, y_res_train):
        if y_ac == y_pred:
            correct = correct + 1
        total = total + 1
    print('Train Accuracy: ' + str(correct) + str('/') + str(total) + str(' = ') + str(correct/total*100)) 
    
    
    true1, true2, true3, true4, true5, true6, true7, true8, true9, true0 = 0,0, 0,0,0,0,0,0,0,0
    tot1, tot2, tot3, tot4, tot5, tot6, tot7, tot8, tot9, tot0 = 0,0, 0,0,0,0,0,0,0,0

    for y_ac, y_pred in zip(y_test, y_res):
        if y_ac == 0:
            tot0 = tot0+1
            if y_ac == y_pred:
                true0 = true0 + 1
        if y_ac == 1:
            tot1 = tot1+1
            if y_ac == y_pred:
                true1 = true1 + 1

        if y_ac == 2:
            tot2 = tot2+1
            if y_ac == y_pred:
                true2 = true2 + 1
        if y_ac == 3:
            tot3 = tot3+1
            if y_ac == y_pred:
                true3 = true3 + 1
        if y_ac == 4:
            tot4 = tot4+1
            if y_ac == y_pred:
                true4 = true4 + 1

        if y_ac == 0:
            tot5 = tot5+1
            if y_ac == y_pred:
                true5 = true5 + 1
        if y_ac == 6:
            tot6 = tot6+1
            if y_ac == y_pred:
                true6 = true6 + 1
        if y_ac == 7:
            tot7 = tot7+1
            if y_ac == y_pred:
                true7 = true7 + 1

        if y_ac == 8:
            tot8 = tot8+1
            if y_ac == y_pred:
                true8 = true8 + 1
        if y_ac == 9:
            tot9 = tot9+1
            if y_ac == y_pred:
                true9 = true9 + 1
                
    tot = tot0 + tot1 +tot2 +tot3 +tot4 +tot5+tot6+tot7+tot8+tot9
    true = true0 + true1 + true2 + true3 + true4 + true5 + true6 + true7 + true8 + true9
    print('Test Accuracy: ' + str(true) + str('/') + str(tot) + str(' = ') + str(true/tot*100))
    print('HV-A Bituminous: ' + str(true0) + str('/') + str(tot0) + str(' = ') + str(true0/tot0*100))
    print('HV-B Bituminous: ' + str(true1) + str('/') + str(tot1) + str(' = ') + str(true1/tot1*100))
    print('HV-C Bituminous: ' + str(true2) + str('/') + str(tot2) + str(' = ') + str(true2/tot2*100))
    print('LV Bituminous: ' + str(true3) + str('/') + str(tot3) + str(' = ') + str(true3/tot3*100))
    print('Lignite-A: ' + str(true4) + str('/') + str(tot4) + str(' = ') + str(true4/tot4*100))
    print('MV Bituminous: ' + str(true5) + str('/') + str(tot5) + str(' = ') + str(true5/tot5*100))
    print('Semi-anthracite: ' + str(true6) + str('/') + str(tot6) + str(' = ') + str(true6/tot6*100))
    print('SubBituminous-A: ' + str(true7) + str('/') + str(tot7) + str(' = ') + str(true7/tot7*100))
    print('SubBituminous-B: ' + str(true8) + str('/') + str(tot8) + str(' = ') + str(true8/tot8*100))
    print('SubBituminous-C: ' + str(true9) + str('/') + str(tot9) + str(' = ') + str(true9/tot9*100))


In [8]:
def logistic_regression(df1, df_id, class_weight = None, impute = True):
    
    enc = LabelEncoder()
    y = df1.iloc[:,-1:].values
    enc.fit(y)
    y = enc.transform(y)
    print(list(enc.classes_))
    X = StandardScaler().fit_transform(df1.iloc[:,1:-1])
    X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.2)
    if impute:
        oversampler= sv.MulticlassOversampling(sv.distance_SMOTE(n_neighbors = 10))
        X_train, y_train = oversampler.sample(X_train, y_train)

#         adsn = ADASYN(k=10,imb_threshold=0.5, ratio=0.75)
#         X_train, y_train = adsn.fit_transform(X_train, y_train)

    skf = StratifiedKFold(n_splits=5)
    if impute:
        pca = PCA(n_components = 500)
        x_train = pca.fit_transform(X_train)
        x_test = pca.transform(X_test)
    else:
        print(X_train.shape[0])
        pca = PCA(n_components = X_train.shape[0])
        x_train = pca.fit_transform(X_train)
        x_test = pca.transform(X_test)
        
    clf = LogisticRegression(class_weight = class_weight)
    model = clf
    model.fit(x_train, y_train)
    y_res = model.predict(x_test)
    
    y_res_train = model.predict(x_train)
    total, correct = 0,0
    for y_ac, y_pred in zip(y_test, y_res):
        if y_ac == y_pred:
            correct = correct + 1
        total = total + 1
    print('Accuracy: ' + str(correct) + str('/') + str(total) + str(' = ') + str(correct/total*100)) 
    
    
#     true1, true2, true3, true4, true5, true6, true7, true8, true9, true0 = 0,0, 0,0,0,0,0,0,0,0
#     tot1, tot2, tot3, tot4, tot5, tot6, tot7, tot8, tot9, tot0 = 0,0, 0,0,0,0,0,0,0,0

#     for y_ac, y_pred in zip(y_test, y_res):
#         if y_ac == 0:
#             tot0 = tot0+1
#             if y_ac == y_pred:
#                 true0 = true0 + 1
#         if y_ac == 1:
#             tot1 = tot1+1
#             if y_ac == y_pred:
#                 true1 = true1 + 1

#         if y_ac == 2:
#             tot2 = tot2+1
#             if y_ac == y_pred:
#                 true2 = true2 + 1
#         if y_ac == 3:
#             tot3 = tot3+1
#             if y_ac == y_pred:
#                 true3 = true3 + 1
#         if y_ac == 4:
#             tot4 = tot4+1
#             if y_ac == y_pred:
#                 true4 = true4 + 1

#         if y_ac == 0:
#             tot5 = tot5+1
#             if y_ac == y_pred:
#                 true5 = true5 + 1
#         if y_ac == 6:
#             tot6 = tot6+1
#             if y_ac == y_pred:
#                 true6 = true6 + 1
#         if y_ac == 7:
#             tot7 = tot7+1
#             if y_ac == y_pred:
#                 true7 = true7 + 1

#         if y_ac == 8:
#             tot8 = tot8+1
#             if y_ac == y_pred:
#                 true8 = true8 + 1
#         if y_ac == 9:
#             tot9 = tot9+1
#             if y_ac == y_pred:
#                 true9 = true9 + 1
                
#     tot = tot0 + tot1 +tot2 +tot3 +tot4 +tot5+tot6+tot7+tot8+tot9
#     true = true0 + true1 + true2 + true3 + true4 + true5 + true6 + true7 + true8 + true9
#     print('Test Accuracy: ' + str(true) + str('/') + str(tot) + str(' = ') + str(true/tot*100))
#     print('HV-A Bituminous: ' + str(true0) + str('/') + str(tot0) + str(' = ') + str(true0/tot0*100))
#     print('HV-B Bituminous: ' + str(true1) + str('/') + str(tot1) + str(' = ') + str(true1/tot1*100))
#     print('HV-C Bituminous: ' + str(true2) + str('/') + str(tot2) + str(' = ') + str(true2/tot2*100))
#     print('LV Bituminous: ' + str(true3) + str('/') + str(tot3) + str(' = ') + str(true3/tot3*100))
#     print('Lignite-A: ' + str(true4) + str('/') + str(tot4) + str(' = ') + str(true4/tot4*100))
#     print('MV Bituminous: ' + str(true5) + str('/') + str(tot5) + str(' = ') + str(true5/tot5*100))
#     print('Semi-anthracite: ' + str(true6) + str('/') + str(tot6) + str(' = ') + str(true6/tot6*100))
#     print('SubBituminous-A: ' + str(true7) + str('/') + str(tot7) + str(' = ') + str(true7/tot7*100))
#     print('SubBituminous-B: ' + str(true8) + str('/') + str(tot8) + str(' = ') + str(true8/tot8*100))
#     print('SubBituminous-C: ' + str(true9) + str('/') + str(tot9) + str(' = ') + str(true9/tot9*100))
    temp_np1 = np.vstack((y_train, y_res_train))
    temp_np2 = np.vstack((y_test, y_res))
    temp_np = np.hstack((temp_np1, temp_np2))
    df_temp = pd.DataFrame(temp_np2)
    filename = str(df_id) + '_' + 'LR_' + str(impute) + str(round(correct/total*100,2))+ '.csv'
    df_temp.to_csv('./results/' + filename,index=False)


In [45]:
for i in range(50):
    #SVM(dfTotal, df_id = i, impute = True, class_weight = 'balanced')
    SVM(dfTotal, df_id = i, impute = False, class_weight = 'balanced')
    i = i + 1

['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
{'C': 100, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 29/43 = 67.44186046511628
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
{'C': 100, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 25/43 = 58.139534883720934
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
{'C': 100, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 29/43 = 67.44186046511628
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
{'C': 0.1, 'gamma': 

{'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 16/43 = 37.2093023255814
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
{'C': 100, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 29/43 = 67.44186046511628
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
{'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 20/43 = 46.51162790697674
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
{'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 17/43 = 39.53488372093023
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthrac

In [12]:
for i in range(50):
    #random_forest(dfTotal, df_id = i, impute = True, class_weight = 'balanced')
    random_forest(dfTotal, df_id = i, impute = False, class_weight = 'balanced')
    i = i + 1

['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
169
{'max_depth': 20, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 1000}
Accuracy: 29/43 = 67.44186046511628
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
169
{'max_depth': 5, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 1000}
Accuracy: 27/43 = 62.7906976744186
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
169
{'max_depth': 5, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 2000}
Accuracy: 24/43 = 55.81395348837209
['HV-A Bituminou

{'max_depth': 5, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 2000}
Accuracy: 26/43 = 60.46511627906976
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
169
{'max_depth': 5, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 1000}
Accuracy: 28/43 = 65.11627906976744
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
169
{'max_depth': 10, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 1000}
Accuracy: 28/43 = 65.11627906976744
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
169
{'max_depth': 5

In [48]:
for i in range(50):
    #logistic_regression(dfTotal, df_id = i, impute = True, class_weight = 'balanced')
    logistic_regression(dfTotal, df_id = i, impute = False, class_weight = 'balanced')
    i = i + 1

['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
169
Accuracy: 19/43 = 44.18604651162791
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
169
Accuracy: 17/43 = 39.53488372093023
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
169
Accuracy: 20/43 = 46.51162790697674
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
169
Accuracy: 23/43 = 53.48837209302325
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthr

['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
169
Accuracy: 18/43 = 41.86046511627907
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
169
Accuracy: 24/43 = 55.81395348837209
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
169
Accuracy: 21/43 = 48.837209302325576
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']
169
Accuracy: 22/43 = 51.162790697674424
['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-ant

In [17]:
#xls = pd.ExcelFile('Reflectance-rank-Test training 09.06.2020.xlsx')
xls = pd.ExcelFile('Reflectance-broad rank-Test training 25.06.2020.xlsx')


dfLignite = pd.read_excel(xls, 'Lignite')
dfLignite = dfLignite.iloc[:,:-2]
y = []
for i in range(len(dfLignite)):
    y.append('Lignite')
dfLignite = dfLignite.assign(label = y)

dfSubbituminous = pd.read_excel(xls, 'Subbituminous')
dfSubbituminous = dfSubbituminous.iloc[:,:-2]
y = []
for i in range(len(dfSubbituminous)):
    y.append('Subbituminous')
dfSubbituminous = dfSubbituminous.assign(label = y)

dfBituminous = pd.read_excel(xls, 'Bituminous')
dfBituminous = dfBituminous.iloc[:,:-2]
y = []
for i in range(len(dfBituminous)):
    y.append('Bituminous')
dfBituminous = dfBituminous.assign(label = y)


dfSemianthracite = pd.read_excel(xls, 'Semi-anthracite')
dfSemianthracite = dfSemianthracite.iloc[:,:-2]
y = []
for i in range(len(dfSemianthracite)):
    y.append('Semianthracite')
dfSemianthracite = dfSemianthracite.assign(label = y)
dfTotalBroader = pd.concat([dfLignite, dfSubbituminous], axis = 0)
dfTotalBroader = pd.concat([dfTotalBroader, dfBituminous], axis = 0)
dfTotalBroader = pd.concat([dfTotalBroader, dfSemianthracite], axis = 0)
from sklearn.utils import shuffle
dfTotalBroader = shuffle(dfTotalBroader)
(unique, counts) = np.unique(dfTotalBroader.iloc[:,-1], return_counts = True)
for u,c in zip(unique, counts):
    print(u,c)

Bituminous 181
Lignite 6
Semianthracite 6
Subbituminous 19


In [19]:
for i in range(50):
    #logistic_regression(dfTotalBroader, df_id = i, impute = True, class_weight = 'balanced')
    logistic_regression(dfTotalBroader, df_id = i, impute = False, class_weight = 'balanced')
    i = i + 1

['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
Accuracy: 35/43 = 81.3953488372093
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
Accuracy: 37/43 = 86.04651162790698
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
Accuracy: 33/43 = 76.74418604651163
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
Accuracy: 36/43 = 83.72093023255815
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
Accuracy: 35/43 = 81.3953488372093
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
Accuracy: 33/43 = 76.74418604651163
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
Accuracy: 32/43 = 74.4186046511628
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
Accuracy: 30/43 = 69.76744186046511
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
Accuracy: 38/43 = 88.37209302325581
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
Accuracy: 31/43 = 72.09302325

In [20]:
for i in range(60):
    #random_forest(dfTotalBroader, df_id = i, impute = True, class_weight = 'balanced')
    random_forest(dfTotalBroader, df_id = i, impute = False, class_weight = 'balanced')
    i = i + 1

['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
{'max_depth': 20, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 1000}
Accuracy: 41/43 = 95.34883720930233
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
{'max_depth': 5, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 1000}
Accuracy: 42/43 = 97.67441860465115
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
{'max_depth': 20, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 1000}
Accuracy: 41/43 = 95.34883720930233
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
{'max_depth': 20, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 1000}
Accuracy: 41/43 = 95.34883720930233
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
{'max_depth': 20, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators'

{'max_depth': 20, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 1000}
Accuracy: 41/43 = 95.34883720930233
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
{'max_depth': 5, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 1000}
Accuracy: 41/43 = 95.34883720930233
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
{'max_depth': 20, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 2000}
Accuracy: 41/43 = 95.34883720930233
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
{'max_depth': 20, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 1000}
Accuracy: 40/43 = 93.02325581395348
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
169
{'max_depth': 20, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 1000}
Accuracy: 40/43 = 93.02325581395348
['Bituminous', 'Ligni

In [21]:
for i in range(60):
    #SVM(dfTotalBroader, df_id = i, impute = True, class_weight = 'balanced')
    SVM(dfTotalBroader, df_id = i, impute = False, class_weight = 'balanced')
    i = i + 1

['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 35/43 = 81.3953488372093
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 38/43 = 88.37209302325581
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
Accuracy: 31/43 = 72.09302325581395
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 31/43 = 72.09302325581395
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
{'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
Accuracy: 35/43 = 81.3953488372093
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 37/43 = 86.04651162790698
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
{'C': 200, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 35/43 = 81.3953488372093
['Bitum

{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 38/43 = 88.37209302325581
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
Accuracy: 35/43 = 81.3953488372093
['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']
{'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 31/43 = 72.09302325581395


In [3]:
rf_all = pd.read_csv('all_class_rf_best.csv')
svm_all = pd.read_csv('all_class_svm_best.csv')
lr_all = pd.read_csv('all_class_lr_best.csv')
target_names = ['HV-A Bituminous', 'HV-B Bituminous', 'HV-C Bituminous', 'LV Bituminous', 'Lignite-A', 'MV Bituminous', 'Semi-anthracite', 'Subbituminous-A', 'Subbituminous-B', 'Subbituminous-C']

In [4]:
def get_report(df, target_names):
    y_true, y_pred = np.asarray(df.iloc[0]), np.asarray(df.iloc[1])
    count, acc = 0, 0
    for t,p in zip(y_true, y_pred):
        if t == p:
            acc = acc + 1
        count = count + 1
    #print(acc, count, acc/count * 100)
    print(classification_report(y_true, y_pred, target_names=target_names))

In [5]:
get_report(rf_all, target_names)

                 precision    recall  f1-score   support

HV-A Bituminous       0.80      0.67      0.73         6
HV-B Bituminous       0.00      0.00      0.00         1
HV-C Bituminous       1.00      1.00      1.00         2
  LV Bituminous       0.70      1.00      0.82        21
      Lignite-A       1.00      1.00      1.00         1
  MV Bituminous       1.00      0.14      0.25         7
Semi-anthracite       0.00      0.00      0.00         1
Subbituminous-A       1.00      1.00      1.00         2
Subbituminous-B       0.00      0.00      0.00         1
Subbituminous-C       0.50      1.00      0.67         1

       accuracy                           0.74        43
      macro avg       0.60      0.58      0.55        43
   weighted avg       0.74      0.74      0.68        43



In [6]:
get_report(svm_all, target_names)

                 precision    recall  f1-score   support

HV-A Bituminous       0.86      1.00      0.92         6
HV-B Bituminous       1.00      1.00      1.00         1
HV-C Bituminous       0.50      1.00      0.67         2
  LV Bituminous       0.89      0.81      0.85        21
      Lignite-A       1.00      1.00      1.00         1
  MV Bituminous       0.50      0.43      0.46         7
Semi-anthracite       0.50      1.00      0.67         1
Subbituminous-A       0.67      1.00      0.80         2
Subbituminous-B       0.00      0.00      0.00         1
Subbituminous-C       0.00      0.00      0.00         1

       accuracy                           0.77        43
      macro avg       0.59      0.72      0.64        43
   weighted avg       0.75      0.77      0.75        43



In [7]:
get_report(lr_all, target_names)

                 precision    recall  f1-score   support

HV-A Bituminous       0.83      0.83      0.83         6
HV-B Bituminous       0.00      0.00      0.00         1
HV-C Bituminous       0.00      0.00      0.00         2
  LV Bituminous       0.81      0.81      0.81        21
      Lignite-A       0.00      0.00      0.00         1
  MV Bituminous       0.71      0.71      0.71         7
Semi-anthracite       0.00      0.00      0.00         1
Subbituminous-A       0.50      1.00      0.67         2
Subbituminous-B       0.00      0.00      0.00         1
Subbituminous-C       1.00      1.00      1.00         1

       accuracy                           0.70        43
      macro avg       0.39      0.44      0.40        43
   weighted avg       0.67      0.70      0.68        43



In [9]:
rf_broader = pd.read_csv('broader_rf_best.csv')
svm_broader = pd.read_csv('broader_svm_best.csv')
lr_broader = pd.read_csv('broader_lr_best.csv')
target_names = ['Bituminous', 'Lignite', 'Semianthracite', 'Subbituminous']

In [10]:
get_report(lr_broader, target_names)

                precision    recall  f1-score   support

    Bituminous       0.97      0.95      0.96        37
       Lignite       0.50      1.00      0.67         1
Semianthracite       0.50      1.00      0.67         1
 Subbituminous       0.67      0.50      0.57         4

      accuracy                           0.91        43
     macro avg       0.66      0.86      0.72        43
  weighted avg       0.92      0.91      0.91        43



In [13]:
get_report(rf_broader, target_names)

                precision    recall  f1-score   support

    Bituminous       0.97      1.00      0.99        37
       Lignite       1.00      1.00      1.00         1
Semianthracite       0.00      0.00      0.00         1
 Subbituminous       1.00      1.00      1.00         4

      accuracy                           0.98        43
     macro avg       0.74      0.75      0.75        43
  weighted avg       0.95      0.98      0.97        43



In [14]:
get_report(svm_broader, target_names)

                precision    recall  f1-score   support

    Bituminous       0.97      1.00      0.99        37
       Lignite       1.00      1.00      1.00         1
Semianthracite       0.00      0.00      0.00         1
 Subbituminous       1.00      1.00      1.00         4

      accuracy                           0.98        43
     macro avg       0.74      0.75      0.75        43
  weighted avg       0.95      0.98      0.97        43

